In [ ]:
# ==========================================
# Step 1: Environment Setup
# ==========================================
!pip install transformers datasets accelerate torch torchvision clean-text matplotlib seaborn

import os
import json
import zipfile
import torch
import random
import string
import collections
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from transformers import BlipProcessor, BlipForQuestionAnswering
from tqdm.auto import tqdm
from google.colab import drive

# Force clear GPU memory
gc.collect()
torch.cuda.empty_cache()

drive.mount('/content/drive')

# --- Parameter Configuration ---
ZIP_PATH = '/content/drive/MyDrive/MV_Project/Slake.zip'
EXTRACT_PATH = '/content/slake_final_viz_best'
CHECKPOINT_PATH = '/content/drive/MyDrive/MV_Project/blip_final_best_v8.pth'

# --- Core Decision: Revert to 4x8 for maximum stability ---
BATCH_SIZE = 4            # Physical Batch (Prevent VRAM OOM)
GRAD_ACCUM_STEPS = 8      # [Revert to 8] Virtual Batch = 32 (Gold Standard)
LEARNING_RATE = 2e-5      # Fine-tuning learning rate
NUM_EPOCHS = 15
PATIENCE = 5
BASELINE_ACC = 73.8

print("[INFO] Environment setup completed.")

In [ ]:
# ==========================================
# Step 2: Data Visualization
# ==========================================
if not os.path.exists(EXTRACT_PATH):
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(EXTRACT_PATH)

img_dir = None
json_path = None
for root, dirs, files in os.walk(EXTRACT_PATH):
    if '__MACOSX' in root: continue
    if 'imgs' in dirs: img_dir = os.path.join(root, 'imgs')
    elif 'images' in dirs: img_dir = os.path.join(root, 'images')
    for file in files:
        if file == 'train.json': json_path = os.path.join(root, file)

print(f"[INFO] JSON Path: {json_path}")

def visualize_dataset(json_file):
    print("[INFO] Generating Data Plots...")
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    en_data = [x for x in data if x.get('q_lang') == 'en']

    q_types = []
    answers = []
    for item in en_data:
        ans = str(item['answer']).lower().strip()
        answers.append(ans)
        if ans in ['yes', 'no']:
            q_types.append('Closed (Yes/No)')
        else:
            q_types.append('Open-Ended')

    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    type_counts = collections.Counter(q_types)
    plt.pie(type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%',
            colors=['#66b3ff','#ff9999'], explode=(0.05, 0))
    plt.title('Question Type Distribution')

    plt.subplot(1, 2, 2)
    ans_counts = collections.Counter(answers).most_common(10)
    sns.barplot(x=[x[1] for x in ans_counts], y=[x[0] for x in ans_counts], palette='viridis')
    plt.title('Top 10 Answers')
    plt.xlabel('Count')

    plt.tight_layout()
    plt.show()

visualize_dataset(json_path)

In [ ]:
# ==========================================
# Extra Feature: Visualize Samples
# ==========================================

def show_data_samples(json_file, img_dir, num_samples=3):
    print(f"[INFO] Displaying {num_samples} Random Samples from Dataset...")

    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    # Filter only English data
    en_data = [x for x in data if x.get('q_lang') == 'en']

    # Random sampling
    samples = random.sample(en_data, num_samples)

    plt.figure(figsize=(15, 5))

    for i, item in enumerate(samples):
        img_path = os.path.join(img_dir, item['img_name'])
        question = item['question']
        answer = item['answer']
        q_type = "Closed" if str(answer).lower() in ['yes', 'no'] else "Open"

        try:
            image = Image.open(img_path).convert('RGB')

            plt.subplot(1, num_samples, i+1)
            plt.imshow(image)
            plt.axis('off')

            # Display Q&A above the image
            # Note: textwrap helps prevent questions from exceeding image width (logic not strictly implemented here but recommended)
            title_text = f"Q: {question}\nA: {answer} ({q_type})"
            plt.title(title_text, fontsize=12, color='blue', loc='left')

        except Exception as e:
            print(f"Cannot load image {img_path}: {e}")

    plt.tight_layout()
    plt.show()

# Run visualization
show_data_samples(json_path, img_dir, num_samples=4)

In [ ]:
# ==========================================
# Step 3: Dataset & Metrics
# ==========================================
def normalize_text(text):
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.strip()

def compute_f1_score(pred, truth):
    pred_tokens = normalize_text(pred).split()
    truth_tokens = normalize_text(truth).split()
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    common = collections.Counter(pred_tokens) & collections.Counter(truth_tokens)
    num_same = sum(common.values())
    if num_same == 0: return 0
    precision = 1.0 * num_same / len(pred_tokens)
    recall = 1.0 * num_same / len(truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def compute_accuracy_robust(pred, truth, q_type):
    p_norm = normalize_text(pred)
    t_norm = normalize_text(truth)
    if q_type == 'closed':
        if t_norm == 'yes' and 'yes' in p_norm: return 1
        if t_norm == 'no' and 'no' in p_norm: return 1
        return 0
    if t_norm in p_norm: return 1
    if p_norm in t_norm and len(p_norm) > 2: return 1
    return 0

class SlakeDataset(Dataset):
    def __init__(self, json_path, img_dir, processor):
        with open(json_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.en_data = [x for x in self.data if x.get('q_lang') == 'en']
        self.img_dir = img_dir
        self.processor = processor

    def __len__(self):
        return len(self.en_data)

    def __getitem__(self, idx):
        item = self.en_data[idx]
        img_path = os.path.join(self.img_dir, item['img_name'])
        try:
            image = Image.open(img_path).convert('RGB')
        except:
            image = Image.new('RGB', (224, 224))

        question = item['question']
        raw_answer = item['answer']
        answer = str(raw_answer).lower()

        text_input = f"Question: {question} Answer:"
        q_type = "closed" if normalize_text(answer) in ['yes', 'no'] else "open"

        encoding = self.processor(image, text_input, padding="max_length", truncation=True, return_tensors="pt")
        labels = self.processor.tokenizer(
            answer, max_length=8, padding="max_length", truncation=True, return_tensors='pt'
        ).input_ids

        return {
            "pixel_values": encoding.pixel_values.squeeze(),
            "input_ids": encoding.input_ids.squeeze(),
            "labels": labels.squeeze(),
            "answer_text": raw_answer,
            "q_type": q_type
        }
print("[INFO] Dataset metrics ready.")

In [ ]:
# ==========================================
# Step 4: Model Loading (Model & Freeze)
# ==========================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Loading BLIP on {device}...")

processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)

# --- Freeze Vision Layers ---
for param in model.vision_model.parameters():
    param.requires_grad = False
print("[INFO] Vision Encoder frozen.")

optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE)

full_dataset = SlakeDataset(json_path, img_dir, processor)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_set, test_set = torch.utils.data.random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

print(f"[INFO] Train: {len(train_set)}, Test: {len(test_set)}")

In [ ]:
# ==========================================
# Step 5: Training Loop - 4x8
# ==========================================
def evaluate_metrics(loader):
    model.eval()
    t_acc, t_f1, open_c, open_t, closed_c, closed_t, t_loss, cnt = 0, 0, 0, 0, 0, 0, 0, 0
    with torch.no_grad():
        for batch in tqdm(loader, desc="Eval", leave=False):
            p_v = batch['pixel_values'].to(device)
            i_i = batch['input_ids'].to(device)
            lbl = batch['labels'].to(device)
            ans = batch['answer_text']
            typ = batch['q_type']

            out = model(input_ids=i_i, pixel_values=p_v, labels=lbl)
            t_loss += out.loss.item() * len(ans)

            gen = model.generate(pixel_values=p_v, input_ids=i_i, max_new_tokens=10)
            preds = processor.batch_decode(gen, skip_special_tokens=True)

            for p, t, qt in zip(preds, ans, typ):
                acc = compute_accuracy_robust(p, t, qt)
                f1 = compute_f1_score(p, t)
                t_acc += acc; t_f1 += f1
                if qt == 'closed': closed_c += acc; closed_t += 1
                else: open_c += acc; open_t += 1
            cnt += len(ans)

    return (t_loss/cnt, t_acc/cnt*100, t_f1/cnt,
            open_c/open_t*100 if open_t>0 else 0,
            closed_c/closed_t*100 if closed_t>0 else 0)

history = {k: [] for k in ['tr_loss','te_loss','te_acc','te_f1','te_o_acc','te_c_acc']}
best_acc = 0.0
patience_cnt = 0

print(f"[INFO] Start Training (Virtual Batch = {BATCH_SIZE * GRAD_ACCUM_STEPS})...")

for epoch in range(NUM_EPOCHS):
    model.train()
    optimizer.zero_grad()
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    tr_loss_acc = 0
    steps = 0

    for idx, batch in enumerate(loop):
        p_v = batch['pixel_values'].to(device)
        i_i = batch['input_ids'].to(device)
        lbl = batch['labels'].to(device)

        out = model(input_ids=i_i, pixel_values=p_v, labels=lbl)
        loss = out.loss / GRAD_ACCUM_STEPS
        loss.backward()
        tr_loss_acc += loss.item() * GRAD_ACCUM_STEPS
        steps += 1

        if (idx + 1) % GRAD_ACCUM_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
            loop.set_postfix(loss=loss.item() * GRAD_ACCUM_STEPS)

    tr_loss = tr_loss_acc / steps
    te_loss, te_acc, te_f1, te_o, te_c = evaluate_metrics(test_loader)

    history['tr_loss'].append(tr_loss); history['te_loss'].append(te_loss)
    history['te_acc'].append(te_acc); history['te_f1'].append(te_f1)
    history['te_o_acc'].append(te_o); history['te_c_acc'].append(te_c)

    print(f"\n[Epoch {epoch+1}] Acc: {te_acc:.2f}% (Best: {best_acc:.2f}%) | F1: {te_f1:.4f}")
    print(f"Open: {te_o:.2f}% | Closed: {te_c:.2f}% | Loss: {te_loss:.4f}")

    if te_acc > best_acc:
        best_acc = te_acc
        patience_cnt = 0
        torch.save(model.state_dict(), CHECKPOINT_PATH)
        print(">>> New Best Model Saved!")
    else:
        patience_cnt += 1
        if patience_cnt >= PATIENCE:
            print(">>> Early Stopping!")
            break

In [ ]:
# ==========================================
# Step 6: Final Visualization & Report (Viz & Report)
# ==========================================
import matplotlib.pyplot as plt
import numpy as np

# --- Part 1: Generate 9 Plots (3x3 Grid) ---
print("[INFO] Generating 3x3 Analytical Plots...")

plt.style.use('seaborn-v0_8-whitegrid')
fig, axes = plt.subplots(3, 3, figsize=(20, 15))
epochs = range(1, len(history['total_acc_te']) + 1)

# Helper Plotting Function
def plot_ax(ax, tr_data, te_data, title, ylabel, baseline=None, color_te='tab:blue'):
    ax.plot(epochs, tr_data, label='Train', marker='.', linestyle='--', color='gray', alpha=0.6)
    ax.plot(epochs, te_data, label='Test (Ours)', marker='o', linewidth=2, color=color_te)
    if baseline:
        ax.axhline(y=baseline, color='tab:red', linestyle='--', linewidth=2, label=f'Baseline ({baseline}%)')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.set_xlabel('Epochs')
    ax.set_ylabel(ylabel)
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)

# --- Row 1: Loss ---
plot_ax(axes[0,0], history['total_loss_tr'], history['total_loss_te'], 'Total Loss Convergence', 'Loss', color_te='tab:blue')
plot_ax(axes[0,1], history['open_loss_tr'], history['open_loss_te'], 'Open-Ended Loss', 'Loss', color_te='tab:green')
plot_ax(axes[0,2], history['closed_loss_tr'], history['closed_loss_te'], 'Closed-Ended Loss', 'Loss', color_te='tab:orange')

# --- Row 2: Accuracy ---
plot_ax(axes[1,0], history['total_acc_tr'], history['total_acc_te'], 'Total Accuracy vs Baseline', 'Accuracy (%)', baseline=BASELINE_ACC, color_te='tab:blue')
plot_ax(axes[1,1], history['open_acc_tr'], history['open_acc_te'], 'Open-Ended Accuracy', 'Accuracy (%)', color_te='tab:green')
plot_ax(axes[1,2], history['closed_acc_tr'], history['closed_acc_te'], 'Closed-Ended Accuracy', 'Accuracy (%)', color_te='tab:orange')

# --- Row 3: F1 Score (Semantic Understanding) ---
plot_ax(axes[2,0], history['total_f1_tr'], history['total_f1_te'], 'Total F1 Score', 'F1 Score', color_te='tab:purple')
plot_ax(axes[2,1], history['open_f1_tr'], history['open_f1_te'], 'Open-Ended F1', 'F1 Score', color_te='tab:purple')
plot_ax(axes[2,2], history['closed_f1_tr'], history['closed_f1_te'], 'Closed-Ended F1', 'F1 Score', color_te='tab:purple')

plt.tight_layout()
plt.show()

# --- Part 2: Generate Text Report ---
print("\n" + "="*80)
print("             FINAL EXPERIMENTAL REPORT: MED-VQA (BLIP OPTIMIZED)")
print("="*80)

# 1. Experimental Setup Overview
print(f"1. EXPERIMENTAL SETUP")
print(f"   - Model Architecture    : BLIP (Salesforce/blip-vqa-base)")
print(f"   - Vision Encoder        : Frozen (Pre-trained weights locked)")
print(f"   - Optimization Strategy : Gradient Accumulation (Steps={GRAD_ACCUM_STEPS})")
print(f"   - Effective Batch Size  : {BATCH_SIZE * GRAD_ACCUM_STEPS} (Physical: {BATCH_SIZE})")
print(f"   - Total Epochs Trained  : {len(history['total_acc_te'])}")
print("-" * 80)

# 2. Final Core Metrics
final_acc = best_acc
final_f1 = history['total_f1_te'][-1]
final_open = history['open_acc_te'][-1]
final_closed = history['closed_acc_te'][-1]

print(f"2. PERFORMANCE METRICS (TEST SET)")
print(f"   ------------------------------------------------------------")
print(f"   | Metric                | Value       | vs. Baseline ({BASELINE_ACC}%) |")
print(f"   ------------------------------------------------------------")
print(f"   | Total Accuracy        | {final_acc:.2f}%      | {'+' if final_acc >= BASELINE_ACC else ''}{final_acc - BASELINE_ACC:.2f}%           |")
print(f"   | F1 Score              | {final_f1:.4f}      | N/A                 |")
print(f"   | Open-Ended Accuracy   | {final_open:.2f}%      | (Critical Metric)   |")
print(f"   | Closed-Ended Accuracy | {final_closed:.2f}%      | -                   |")
print(f"   ------------------------------------------------------------")

# 3. Automated Conclusion (for Report Conclusion)
print("-" * 80)
print(f"3. AUTOMATED CONCLUSION")
if final_acc > BASELINE_ACC:
    print(f"   SUCCESS: The optimized BLIP model has successfully outperformed the CNN-LSTM")
    print(f"   baseline by a margin of {final_acc - BASELINE_ACC:.2f}%.")
    print(f"   Key Observation: The high F1 score ({final_f1:.4f}) indicates that the generative")
    print(f"   model effectively understands medical semantics beyond simple classification.")
else:
    print(f"   RESULT: The model achieved comparable performance to the baseline.")
    print(f"   Further optimization (e.g., more data, longer training) may be required.")
print("="*80)